# Unit test

In [11]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ubuntu/varios/skforecast


In [12]:
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
from skforecast.metrics import root_mean_squared_scaled_error
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _predict_and_calculate_metrics_multiseries_one_step_ahead

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y_feature_selection
from skforecast.model_selection.tests.fixtures_model_selection import exog_feature_selection


metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
#metrics = [add_y_train_argument(metric) for metric in metrics]
from skforecast.datasets import fetch_dataset



In [13]:
data = fetch_dataset(name="items_sales", verbose=False)
data['day_of_week'] = data.index.dayofweek
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
data_test = data.loc[end_train:, :].copy()
levels = ["item_1", "item_2", "item_3"]
exog_features = 'day_of_week'

metrics = [
    "mean_absolute_error",
    "mean_squared_error",
    mean_absolute_percentage_error,
    mean_absolute_scaled_error,
    root_mean_squared_scaled_error,
]
steps = 1
initial_train_size = 100
param_grid = {
    "alpha": np.logspace(-3, 3, 1),
}
lags_grid = [3]
param_grid = list(ParameterGrid(param_grid))

forecaster = ForecasterAutoregMultiSeries(
    regressor=Ridge(random_state=678),
    lags=3,
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)

forecaster = ForecasterAutoregMultiVariate(
    regressor=Ridge(random_state=678),
    lags=3,
    steps=1,
    level="item_1",
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)

X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
    forecaster._train_test_split_one_step_ahead(
        series=data.loc[:, levels],
        exog=data.loc[:, exog_features] if exog_features else None,
        initial_train_size=initial_train_size,
    )
)

display(data)
display(X_train)
display(X_test)
display(y_train)
display(y_test)
display(X_train_encoding)
display(X_test_encoding)


results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        return_best        = False,
        verbose            = False,
        show_progress      = True
    )

results_one_step_ahead

,item_1,item_2,item_3,day_of_week
date,,,,
2012-01-01,8.253175,21.047727,19.429739,6
2012-01-02,22.777826,26.578125,28.009863,0
2012-01-03,27.549099,31.751042,32.078922,1
2012-01-04,25.895533,24.567708,27.252276,2
2012-01-05,21.379238,18.191667,20.357737,3
...,...,...,...,...
2014-12-28,17.329233,18.189583,20.586030,6
2014-12-29,19.611623,24.539583,28.127390,0
2014-12-30,18.857026,17.677083,21.555782,1


,item_1_lag_1,item_1_lag_2,item_1_lag_3,item_2_lag_1,item_2_lag_2,item_2_lag_3,item_3_lag_1,item_3_lag_2,item_3_lag_3
date,,,,,,,,,
2012-01-04,1.736038,0.069055,-5.005556,2.936737,1.595459,0.161491,2.495660,1.545940,-0.456666
2012-01-05,1.158317,1.736038,0.069055,1.074182,2.936737,1.595459,1.369118,2.495660,1.545940
2012-01-06,-0.419583,1.158317,1.736038,-0.579053,1.074182,2.936737,-0.240071,1.369118,2.495660
2012-01-07,-0.514822,-0.419583,1.158317,-0.677366,-0.579053,1.074182,-0.351774,-0.240071,1.369118
2012-01-08,-0.714936,-0.514822,-0.419583,-0.237116,-0.677366,-0.579053,-0.780215,-0.351774,-0.240071
...,...,...,...,...,...,...,...,...,...
2012-04-05,0.150486,-0.031549,-1.099432,-0.261695,0.493753,-1.096820,-0.338919,-0.047092,-1.404941
2012-04-06,0.130344,0.150486,-0.031549,-0.256563,-0.261695,0.493753,-0.356970,-0.338919,-0.047092
2012-04-07,-0.306645,0.130344,0.150486,0.220690,-0.256563,-0.261695,0.121229,-0.356970,-0.338919


,item_1_lag_1,item_1_lag_2,item_1_lag_3,item_2_lag_1,item_2_lag_2,item_2_lag_3,item_3_lag_1,item_3_lag_2,item_3_lag_3
date,,,,,,,,,
2012-04-10,-1.600750,-1.602861,-1.378975,-1.225113,-1.463605,-0.277360,-1.310729,-0.144628,-0.422595
2012-04-11,-0.666965,-1.600750,-1.602861,-2.482393,-1.225113,-1.463605,-2.610456,-1.310729,-0.144628
2012-04-12,0.154228,-0.666965,-1.600750,-1.628901,-2.482393,-1.225113,-1.567432,-2.610456,-1.310729
2012-04-13,0.035007,0.154228,-0.666965,-1.589738,-1.628901,-2.482393,-1.302918,-1.567432,-2.610456
2012-04-14,-0.048676,0.035007,0.154228,-1.142465,-1.589738,-1.628901,-2.181238,-1.302918,-1.567432
...,...,...,...,...,...,...,...,...,...
2014-12-28,-1.991623,-2.110374,-1.453578,-1.036049,-0.905324,-0.547722,0.082209,-0.455532,-1.455941
2014-12-29,-1.834570,-1.991623,-2.110374,-0.579593,-1.036049,-0.905324,-0.186787,0.082209,-0.455532
2014-12-30,-1.037150,-1.834570,-1.991623,1.066889,-0.579593,-1.036049,1.573370,-0.186787,0.082209


{1: date
 2012-01-04    1.158317
 2012-01-05   -0.419583
 2012-01-06   -0.514822
 2012-01-07   -0.714936
 2012-01-08   -0.877238
                 ...   
 2012-04-05    0.130344
 2012-04-06   -0.306645
 2012-04-07   -1.378975
 2012-04-08   -1.602861
 2012-04-09   -1.600750
 Freq: D, Name: item_1_step_1, Length: 97, dtype: float64}

{1: date
 2012-04-10   -0.666965
 2012-04-11    0.154228
 2012-04-12    0.035007
 2012-04-13   -0.048676
 2012-04-14   -0.464676
                 ...   
 2014-12-28   -1.834570
 2014-12-29   -1.037150
 2014-12-30   -1.300791
 2014-12-31   -1.348238
 2015-01-01   -4.221854
 Freq: D, Name: item_1_step_1, Length: 997, dtype: float64}

...

date
2012-04-10    item_1
2012-04-11    item_1
2012-04-12    item_1
2012-04-13    item_1
2012-04-14    item_1
               ...  
2014-12-28    item_1
2014-12-29    item_1
2014-12-30    item_1
2014-12-31    item_1
2015-01-01    item_1
Freq: D, Length: 997, dtype: object

/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1787: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

,levels,lags,lags_label,params,mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,mean_absolute_scaled_error,root_mean_squared_scaled_error,alpha
0,[item_1],"[1, 2, 3]","[1, 2, 3]",{'alpha': 0.001},1.509283,3.835749,0.067623,0.999911,0.980093,0.001


In [14]:
data = fetch_dataset(name="items_sales", verbose=False)
data['day_of_week'] = data.index.dayofweek
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
levels = ["item_1", "item_2", "item_3"]
exog_features = ['day_of_week']

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 100
    param_grid = {
        "alpha": np.logspace(-1, 1, 1),
    }
    lags_grid = [3]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = deepcopy(forecaster),
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = deepcopy(forecaster),
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    # Diferenciation must be 0 for this test
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='item_1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:394: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


0


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1787: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1787: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


2


KeyboardInterrupt: 

In [5]:
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import exog

series.index = pd.date_range(start='2024-01-01', periods=len(series), freq='D')
exog.index = pd.date_range(start='2024-01-01', periods=len(exog), freq='D')

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 20
    param_grid = {
        "alpha": np.logspace(-1, 1, 3),
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series,
        exog               = exog,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series,
        exog               = exog,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    # Diferenciation must be 0 for this test
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='l1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:394: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


0


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1787: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1787: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


2


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1787: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


3


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1787: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


In [6]:
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 213
    param_grid = {
        "n_estimators": [10, 20],
        "max_depth": [2, 3],
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = LGBMRegressor(random_state=678, verbose=-1)
forecasters = [
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='item_1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

0


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1787: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


ValueError: All values of series 'id_1002' are NaN.

## _calculate_metrics_multiseries vs _calculate_metrics_multiseries_one_step_ahead

In [15]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _predict_and_calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [73]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False in backtesting_forecaster_multiseries


# Data download
series = fetch_dataset(name="items_sales")
exog = pd.DataFrame(
    {
        'day_of_week': series.index.dayofweek
    },
    index = series.index
)
initial_train_size = 927

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecasters = [
    ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        encoding           = 'ordinal',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = False,
    ),
    ForecasterAutoregMultiVariate(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        steps              = 1,
        level              = 'item_1',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
    )
]

for forecaster in forecasters:
    metrics_backtesting, predictions = backtesting_forecaster_multiseries(
        series=series,
        exog=exog,
        forecaster=forecaster,
        steps=1,
        metric=metrics,
        initial_train_size = initial_train_size,
        refit=False,
        add_aggregated_metric=True,
        show_progress=False
    )

    display(metrics_backtesting)

    (
        X_train,
        y_train,
        X_test,
        y_test,
        X_train_encoding,
        X_test_encoding
    ) = forecaster._train_test_split_one_step_ahead(
            series             = series,
            exog               = exog,
            initial_train_size = initial_train_size,
        )

    metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
        forecaster=forecaster,
        series=series,
        X_train = X_train,
        y_train= y_train,
        X_train_encoding = X_train_encoding,
        X_test = X_test,
        y_test = y_test,
        X_test_encoding = X_test_encoding,
        levels = ['item_1', 'item_2', 'item_3'],
        metrics = metrics,
        add_aggregated_metric = True
    )

    display(metrics_one_step_ahead)



    pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)
    pd.testing.assert_frame_equal(predictions_one_step_ahead, predictions)

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.825973,0.041122,0.541134
1,item_2,2.251201,0.146863,0.955679
2,item_3,3.187688,0.216736,0.860802
3,average,2.088287,0.134907,0.785872
4,weighted_average,2.088287,0.134907,0.785872
5,pooling,2.088287,0.134907,0.825939


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.825973,0.041122,0.541134
1,item_2,2.251201,0.146863,0.955679
2,item_3,3.187688,0.216736,0.860802
3,average,2.088287,0.134907,0.785872
4,weighted_average,2.088287,0.134907,0.785872
5,pooling,2.088287,0.134907,0.825939


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.794891,0.039537,0.520771


TypeError: `series` must be a pandas DataFrame. Got <class 'pandas.core.series.Series'>.